## Part-of-Speech tagging using CRF

This project involves Parts Of Speech (POS) tagging using Conditional Random Fields (CRF) approach to achieve better accuracy than traditional HMM POS tagging techniques. The following steps are followed:

1. Data Preparation.
2. Define Feature Functions.
3. Train a CRF classifier with Grid Search  cross validation to find CRF model with best hyperparameters.
4. Checking accuracy of best CRF model on validation and test sets.
5. Find important state and transition features.

### Data Preparation

In [3]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import sklearn

import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# reading the Treebank tagged sentences
# wsj = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [5]:
# first few tagged sentences
# print(wsj[:2])
# print(len(wsj))

In [6]:
# reading the Brown tagged sentences
brown= list(nltk.corpus.brown.tagged_sents(tagset='universal'))

In [7]:
# first few tagged sentences
print(brown[:2])
print(len(brown))

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [8]:
# reading the conll2000 tagged sentences
# conll2000= list(nltk.corpus.conll2000.tagged_sents(tagset='universal'))

In [9]:
# first few tagged sentences
# print(conll2000[:2])
# print(len(conll2000))

In [10]:
nltk_data =  brown 

In [11]:
print(nltk_data[:2])
print(len(nltk_data))

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [12]:
nltk_data[1]

[('The', 'DET'),
 ('jury', 'NOUN'),
 ('further', 'ADV'),
 ('said', 'VERB'),
 ('in', 'ADP'),
 ('term-end', 'NOUN'),
 ('presentments', 'NOUN'),
 ('that', 'ADP'),
 ('the', 'DET'),
 ('City', 'NOUN'),
 ('Executive', 'ADJ'),
 ('Committee', 'NOUN'),
 (',', '.'),
 ('which', 'DET'),
 ('had', 'VERB'),
 ('over-all', 'ADJ'),
 ('charge', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('election', 'NOUN'),
 (',', '.'),
 ('``', '.'),
 ('deserves', 'VERB'),
 ('the', 'DET'),
 ('praise', 'NOUN'),
 ('and', 'CONJ'),
 ('thanks', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('City', 'NOUN'),
 ('of', 'ADP'),
 ('Atlanta', 'NOUN'),
 ("''", '.'),
 ('for', 'ADP'),
 ('the', 'DET'),
 ('manner', 'NOUN'),
 ('in', 'ADP'),
 ('which', 'DET'),
 ('the', 'DET'),
 ('election', 'NOUN'),
 ('was', 'VERB'),
 ('conducted', 'VERB'),
 ('.', '.')]

In [13]:
# split data into training and validation and test set
train_set,test_set = train_test_split(nltk_data,train_size=0.85,test_size=0.15,random_state=101)
train_set,val_set = train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state=101)

### Define Features for feature function:

The following features are defined:
* word: the word itself.
* pos: POS tag of the word.
* prevword: previous word.
* prevpos: POS tag of previous word.
* prev2word: second previous word to the given word.
* prev2pos: POS tag of second previous word.
* nextword: next word.
* nextpos: POS tag of next word.
* pref_1, pref_2, pref_3, pref_4: 1,2,3 and 4 lenght prefix of the word.
* suff_1, suff_2, suff_3, suff_4: 1,2,3 and 4 lenght suffix of the word.

In [14]:

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


def word_features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word': word,
        'len(word)': len(word),
        'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-4:]': word[-4:],
        'word.lower()': word.lower(),
        'word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word.lower()),
        # 'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word': word1,
            '-1:len(word)': len(word1),
            '-1:word.lower()': word1.lower(),
            '-1:word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word1.lower()),
            '-1:word[:3]': word1[:3],
            '-1:word[:2]': word1[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word.isdigit()': word1.isdigit(),
            # '-1:word.ispunctuation': (word1 in string.punctuation),
        })
    else:
        features['BOS'] = True

    if i > 1:
        word2 = sent[i-2][0]
        features.update({
            '-2:word': word2,
            '-2:len(word)': len(word2),
            '-2:word.lower()': word2.lower(),
            '-2:word[:3]': word2[:3],
            '-2:word[:2]': word2[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word.isdigit()': word2.isdigit(),
            # '-2:word.ispunctuation': (word2 in string.punctuation),
        })

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word': word1,
            '+1:len(word)': len(word1),
            '+1:word.lower()': word1.lower(),
            '+1:word[:3]': word1[:3],
            '+1:word[:2]': word1[:2],
            '+1:word[-3:]': word1[-3:],
            '+1:word[-2:]': word1[-2:],
            '+1:word.isdigit()': word1.isdigit(),
            # '+1:word.ispunctuation': (word1 in string.punctuation),
        })

    else:
        features['EOS'] = True

    if i < len(sent) - 2:
        word2 = sent[i+2][0]
        features.update({
            '+2:word': word2,
            '+2:len(word)': len(word2),
            '+2:word.lower()': word2.lower(),
            '+2:word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', word2.lower()),
            '+2:word[:3]': word2[:3],
            '+2:word[:2]': word2[:2],
            '+2:word[-3:]': word2[-3:],
            '+2:word[-2:]': word2[-2:],
            '+2:word.isdigit()': word2.isdigit(),
            # '+2:word.ispunctuation': (word2 in string.punctuation),
        })

    return features



In [15]:
# let's check if our word feature is working correctly:
print(train_set[0][0:5])

word_features(train_set[0],3)

[('A', 'DET'), ('Newfoundland', 'NOUN'), ('sat', 'VERB'), ('solemnly', 'ADV'), ('beside', 'ADP')]


{'bias': 1.0,
 'word': 'solemnly',
 'len(word)': 8,
 'word[:4]': 'sole',
 'word[:3]': 'sol',
 'word[:2]': 'so',
 'word[-3:]': 'nly',
 'word[-2:]': 'ly',
 'word[-4:]': 'mnly',
 'word.lower()': 'solemnly',
 'word.stemmed': 'solemnl',
 'word.isdigit()': False,
 '-1:word': 'sat',
 '-1:len(word)': 3,
 '-1:word.lower()': 'sat',
 '-1:word.stemmed': 'sat',
 '-1:word[:3]': 'sat',
 '-1:word[:2]': 'sa',
 '-1:word[-3:]': 'sat',
 '-1:word[-2:]': 'at',
 '-1:word.isdigit()': False,
 '-2:word': 'Newfoundland',
 '-2:len(word)': 12,
 '-2:word.lower()': 'newfoundland',
 '-2:word[:3]': 'New',
 '-2:word[:2]': 'Ne',
 '-2:word[-3:]': 'and',
 '-2:word[-2:]': 'nd',
 '-2:word.isdigit()': False,
 '+1:word': 'beside',
 '+1:len(word)': 6,
 '+1:word.lower()': 'beside',
 '+1:word[:3]': 'bes',
 '+1:word[:2]': 'be',
 '+1:word[-3:]': 'ide',
 '+1:word[-2:]': 'de',
 '+1:word.isdigit()': False,
 '+2:word': 'a',
 '+2:len(word)': 1,
 '+2:word.lower()': 'a',
 '+2:word.stemmed': 'a',
 '+2:word[:3]': 'a',
 '+2:word[:2]': 'a',


In [16]:
# defining a few more functions to extract featrues, postags and words from sentences

def sent2features(sent):
    return [word_features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [postag for word, postag in sent]

def sent2tokens(sent):
    return [word for word, postag in sent]    

In [17]:
 # create training, validation and test sets
X_train = [sent2features(s) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_valid = [sent2features(s) for s in val_set]
y_valid = [sent2labels(s) for s in val_set]

X_test = [sent2features(s) for s in test_set]
y_test = [sent2labels(s) for s in test_set]

In [18]:
# check the train set produced
print(X_train[0][0:10])

[{'bias': 1.0, 'word': 'A', 'len(word)': 1, 'word[:4]': 'A', 'word[:3]': 'A', 'word[:2]': 'A', 'word[-3:]': 'A', 'word[-2:]': 'A', 'word[-4:]': 'A', 'word.lower()': 'a', 'word.stemmed': 'a', 'word.isdigit()': False, 'BOS': True, '+1:word': 'Newfoundland', '+1:len(word)': 12, '+1:word.lower()': 'newfoundland', '+1:word[:3]': 'New', '+1:word[:2]': 'Ne', '+1:word[-3:]': 'and', '+1:word[-2:]': 'nd', '+1:word.isdigit()': False, '+2:word': 'sat', '+2:len(word)': 3, '+2:word.lower()': 'sat', '+2:word.stemmed': 'sat', '+2:word[:3]': 'sat', '+2:word[:2]': 'sa', '+2:word[-3:]': 'sat', '+2:word[-2:]': 'at', '+2:word.isdigit()': False}, {'bias': 1.0, 'word': 'Newfoundland', 'len(word)': 12, 'word[:4]': 'Newf', 'word[:3]': 'New', 'word[:2]': 'Ne', 'word[-3:]': 'and', 'word[-2:]': 'nd', 'word[-4:]': 'land', 'word.lower()': 'newfoundland', 'word.stemmed': 'newfoundland', 'word.isdigit()': False, '-1:word': 'A', '-1:len(word)': 1, '-1:word.lower()': 'a', '-1:word.stemmed': 'a', '-1:word[:3]': 'A', '-1

In [19]:
y_train[0][0:10]

['DET', 'NOUN', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', 'PRT', 'DET', 'NOUN']

### 1. Build CRF model

In [20]:
# fitting crf with arbitrary hyperparameters
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True   
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.01, c2=0.1,
    max_iterations=100)

In [21]:
labels = list(crf.classes_)

In [1]:
# make predictions 
y_pred = crf.predict(X_valid)
print(X_valid[0][0:10])
print(y_pred[0][0:10])
metrics.flat_f1_score(y_valid, y_pred,
                      average='weighted', labels=labels)
# test = ['This', 'is', 'a', 'bag']
# print(crf.predict([sent2features(test)]))


NameError: name 'crf' is not defined

In [23]:
# class-wise scores
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_valid, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000     29701
           X      0.864     0.507     0.639       300
         ADJ      0.944     0.937     0.940     16852
         ADP      0.979     0.983     0.981     29177
         ADV      0.948     0.933     0.941     11383
        VERB      0.981     0.980     0.980     37018
         DET      0.993     0.996     0.994     27692
        CONJ      0.996     0.997     0.996      7655
        NOUN      0.974     0.982     0.978     55378
        PRON      0.992     0.987     0.989     10032
         PRT      0.952     0.938     0.945      6074
         NUM      0.988     0.967     0.977      2838

    accuracy                          0.979    234100
   macro avg      0.967     0.934     0.947    234100
weighted avg      0.979     0.979     0.979    234100



In [24]:
test = ['This', 'is', 'a', 'bag']
print(crf.predict([sent2features(test)]))

[['NOUN' 'ADP' 'DET' 'NOUN']]


: 